## OpenAI Agents

- Native to OpenAI ecosystem
- Simple to set up
- Best for tool-use workflows
- Supports Python tools, function tools, data browsing, retrieval

In [ ]:
import os
from openai import AsyncOpenAI
from agents import Agent, Runner, set_default_openai_client, set_tracing_disabled, function_tool
from dotenv import load_dotenv
import asyncio

load_dotenv(override=True)

aval_api_key=os.getenv("AVALAI_API_KEY")
openai_api_key=os.getenv("OPENAI_API_KEY")


In [ ]:
# set up custom client
custom_client = AsyncOpenAI(
    base_url="https://api.avalai.ir/v1",
    api_key=aval_api_key
)

set_default_openai_client(custom_client)
set_tracing_disabled(True)

In [ ]:
# define an agent
agent = Agent(name="Assistant", instructions="You are a helpful assistant")

result = await Runner.run(agent, "Tell me a joke about programming.")
print(result.final_output)


## Multi agent

In [ ]:
instructions1 = "تو یک عامل حرفه‌ای هستی که برای درخواست کارآموزی مهندسی هوش مصنوعی، \
ایمیل‌های رسمی و دقیق می‌نویسد. لحن تو جدی، مودب و کاملاً حرفه‌ای است."

instructions2 = "تو یک عامل مشتاق و خلاق هستی که برای درخواست کارآموزی مهندسی هوش مصنوعی، \
ایمیل‌هایی انرژی‌بخش و الهام‌گر می‌نویسد. لحن تو دوستانه، جذاب و پرانگیزه است."

instructions3 = "تو یک عامل بسیار مشغول هستی که برای درخواست کارآموزی مهندسی هوش مصنوعی، \
ایمیل‌های بسیار کوتاه، شفاف و مستقیم می‌نویسد."

In [ ]:
intern_agent1 = Agent(
        name="عامل حرفه‌ای",
        instructions=instructions1,
        model="gpt-4o-mini"
)

intern_agent2 = Agent(
        name="عامل خلاق و مشتاق",
        instructions=instructions2,
        model="gpt-4o-mini"
)

intern_agent3 = Agent(
        name="عامل کوتاه‌نویس",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [ ]:
result = await Runner.run(intern_agent1, "یک ایمیل برای درخواست کارآموزی مهندسی هوش مصنوعی بنویس.")
print(result.final_output)

In [ ]:
message = "یک ایمیل برای درخواست کارآموزی مهندسی هوش مصنوعی بنویس."


results = await asyncio.gather(
    Runner.run(intern_agent1, message),
    Runner.run(intern_agent2, message),
    Runner.run(intern_agent3, message),
)

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")


In [ ]:
internship_picker = Agent(
    name="internship_picker",
    instructions="از بین گزینه‌های ارائه‌شده، بهترین ایمیل درخواست کارآموزی مهندسی هوش مصنوعی را انتخاب کن. \
خودت را در جای یک مدیر منابع انسانی قرار بده و ایمیلی را انتخاب کن که بیشترین احتمال دارد به آن پاسخ بدهی. \
هیچ توضیحی نده؛ فقط متن همان ایمیل منتخب را بنویس.",
    model="gpt-4o-mini",
)

In [ ]:
message = "یک ایمیل برای درخواست کارآموزی مهندسی هوش مصنوعی بنویس."


results = await asyncio.gather(
    Runner.run(intern_agent1, message),
    Runner.run(intern_agent2, message),
    Runner.run(intern_agent3, message),
)
outputs = [result.final_output for result in results]

emails = "AI internship emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

best = await Runner.run(internship_picker, emails)

print(f"Best AI internship email:\n{best.final_output}")


### Sending Gmail Email via SMTP (App Password)
Create an App Password

- Go to myaccount.google.com
- Turn on **2-Step** Verification
- Search for **App Passwords**
- Create one for “Mail”
- Google will give you a 16-character password → copy it

(Do not share it with anyone.)

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [ ]:
@function_tool
def send_email(subject: str, body: str) -> None:
    """ Send out an email with the given subject and HTML body"""

    sender = os.getenv("EMAIL_SENDER")
    receiver = receiver = os.getenv("EMAIL_RECEIVER")
    app_password = os.getenv("GMAIL_APP_PASSWORD")

    msg = MIMEMultipart()
    msg["From"] = sender
    msg["To"] = receiver
    msg["Subject"] = subject

    msg.attach(MIMEText(body, "html"))

    # Connect to Gmail server
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(sender, app_password)
        server.send_message(msg)

    print("Email sent!")


In [ ]:
send_email("Test", "Salam, khobi?")

In [ ]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first write a subject for the email, then convert the body to HTML. \
Finally, you use the send_email tool to send the email with the subject and HTML body. \
Write the subject and body only in Persian (Farsi)"


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=[send_email],
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


In [ ]:
result = await Runner.run(emailer_agent, "Send this email: this is a test email to see if you can send this message to my Gmail")
print(result.final_output)

In [ ]:
internship_manager_instructions = """
تو یک مدیر کارآموزی در یک شرکت فناوری هستی. وظیفه‌ی تو انتخاب بهترین ایمیل درخواست کارآموزی 
مهندسی هوش مصنوعی از میان گزینه‌های ارائه‌شده است.

تو باید فقط و فقط یک ایمیل را از بین ایمیل‌های داده‌شده انتخاب کنی.
خودت را جای مسئول جذب نیرو قرار بده و ایمیلی را انتخاب کن که بیشترین احتمال دارد به آن پاسخ بدهی.
هیچ توضیحی ارائه نده.

هدف: تنها یک ایمیل را بر اساس قضاوت خودت انتخاب کن؛ ایمیلی که مؤثرتر و حرفه‌ای‌تر است.

انتقال برای ارسال: فقط متن «همان یک ایمیل برگزیده» را به عامل 'Email Manager' ارسال کن.
عامل Email Manager خودش قالب‌بندی و ارسال را انجام می‌دهد.

قوانین مهم:
- فقط یکی از ایمیل‌های ارائه‌شده را انتخاب کن — هرگز خودت ایمیل جدیدی ننویس.
- دقیقاً یک ایمیل را برای Email Manager بفرست — نه بیشتر، نه کمتر.
- خروجی باید فقط متن همان ایمیل منتخب باشد و هیچ متن اضافی دیگری شامل توضیح، تحلیل یا مقدمه نباید وجود داشته باشد.


فقط فقط فقط یک ایمیل برای ارسال انتخاب کن. حق نداری بیشتر از یک ایمیل بفرستی.
"""


In [ ]:
email_manager = Agent(
    name="Email Manager",
    instructions=internship_manager_instructions,
    model="gpt-4o-mini",
    handoffs=[emailer_agent],
)

In [ ]:
message = "یک ایمیل برای درخواست کارآموزی مهندسی هوش مصنوعی بنویس."

results = await asyncio.gather(
    Runner.run(intern_agent1, message),
    Runner.run(intern_agent2, message),
    Runner.run(intern_agent3, message),
)
outputs = [result.final_output for result in results]

emails = "Internship emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

result = await Runner.run(email_manager, emails)
result